In [86]:
FILE_LIST = "files.p"
DATA_DIR = "/run/media/maciej/Nowy/data/json/"
CHOSEN_YEAR = "2011"

In [87]:
import regex
word_pattern = "\p{Letter}+"

Oblicz statystykę występowania bigramów słów, pomijając w tekście wszystkie wyrazy, które nie stanowią słów.

In [136]:
import os, json

import pickle
from tqdm import tqdm
from collections import defaultdict

bigrams = defaultdict(lambda: 0)
unigrams = defaultdict(lambda: 0)

def load_data():    
    files = pickle.load(open(FILE_LIST, 'rb'))
    for file in tqdm(files):
        if file.startswith("judgment"):
            file_path = os.path.join(DATA_DIR, file)

            with open(file_path, 'r') as f:
                data = json.load(f)
                judgments = [x["textContent"] for x in data["items"] if x["judgmentDate"].startswith(CHOSEN_YEAR)]

            for judgment in judgments:
                previous_word = None

                judgment = regex.sub("<.*?>", "", judgment)
                judgment = regex.sub("-\n(\p{Letter}+)", r"\1", judgment)
                
                for match in regex.finditer(word_pattern, judgment):
                    [word] = match.captures()
                    
                    
                    if previous_word:
                        bigrams[(previous_word, word.lower())] += 1
                    unigrams[word.lower()] += 1
                    
                    previous_word = word.lower()
load_data()
print(len(bigrams))

100%|██████████| 68/68 [00:33<00:00,  2.03it/s]

2613738


Korzystając z wzoru na punktową informację wzajemną oblicz tę wartość dla wszystkich par słów. Wykorzystaj statystyki unigramów obliczone w poprzednim zadaniu.

In [128]:
# word_file = "words.p"
# words = pickle.load(open(word_file, "rb"))
# unigrams = dict(words)

In [141]:
import numpy as np

unigram_count = sum(unigrams.values())
bigram_count = sum(bigrams.values())

def get_unigram_probability(unigram):
    return unigrams[unigram] / unigram_count

def get_bigram_probability(bigram):
    return bigrams[bigram] / bigram_count

def get_pmi(bigram):
    x, y = bigram
    return np.log(get_bigram_probability(bigram) / (get_unigram_probability(x) * get_unigram_probability(y)))

In [143]:
bigrams_with_pmi = {
    bigram: get_pmi(bigram) for bigram in bigrams
}

Posortuj bigramy względem malejącej wartości punktowej informacji wzajemnej. Przedstaw 30 pierwszych wyników.



In [154]:
def sort_and_display(bigrams_with_values, top=30):
    sorted_bigrams = sorted(
        bigrams_with_values.items(), 
        key=lambda x: x[1], 
        reverse=True,
    )
    
    for bigram, score in sorted_bigrams[:30]:
        print("{} -> {}".format(" ".join(bigram).ljust(40, " "), score))


In [155]:
sort_and_display(bigrams_with_pmi)


domkowi fińskiemu                        -> 16.6181418079068
walnymi zgromadzeniami                   -> 16.6181418079068
wykład inauguracyjny                     -> 16.6181418079068
reiffeisenlandesbank oberosterreich      -> 16.6181418079068
iur samuela                              -> 16.6181418079068
doprowa dzano                            -> 16.6181418079068
twier dzeniom                            -> 16.6181418079068
przewle kłość                            -> 16.6181418079068
wyni kłej                                -> 16.6181418079068
odszko dowania                           -> 16.6181418079068
naczyniopochodnymi zawrotami             -> 16.6181418079068
zawia domienia                           -> 16.6181418079068
pasierbica przysposobiony                -> 16.6181418079068
lota lotb                                -> 16.6181418079068
directly decisive                        -> 16.6181418079068
dostawiona dorysowana                    -> 16.6181418079068
tole rancją             

Korzystając z wzoru na statystykę logarytmiczną opartą o rozkład dwumienny (G2) sporządź analogiczną listę, jak dla punktowej informacji wzajemnej.


In [153]:
import numpy as np

def get_k(bigram):
    k_11 = bigrams[bigram]
    k_12 = unigrams[bigram[1]]
    k_21 = unigrams[bigram[0]]
    k_22 = bigram_count - k_11 - k_12 - k_21
    
    return np.array(
        [[k_11, k_12],
         [k_21, k_22]]
    )

def H(k):
    """Count Shannon's entropy"""
    N = np.sum(k)
    
    return np.sum(k/N * np.log(k/N + (k== 0)))
    
def get_llr(bigram):
    k = get_k(bigram)
    return 2 * np.sum(k) * (H(k) - H(k.sum(axis=0)) -H(k.sum(axis=1)))

In [150]:
get_llr(("krwinek", "czerwonych"))

227.65528248394048

In [152]:
# Dict comprehension switched with tqdm loop in order to watch the progress
# bigrams_with_llr = {
#     bigram: get_llr(bigram) for bigram in bigrams
# }

bigrams_with_llr = {}
for bigram in tqdm(bigrams):
    bigrams_with_llr[bigram] = get_llr(bigram)


100%|██████████| 2613738/2613738 [02:23<00:00, 18226.14it/s]


In [156]:
sort_and_display(bigrams_with_llr)

art ust                                  -> 482467.2308204672
z dnia                                   -> 311233.5602716225
k p                                      -> 265786.0055493093
art k                                    -> 254143.54965792052
nr poz                                   -> 213003.3228910818
p c                                      -> 206885.91223860395
sygn akt                                 -> 204801.9598722935
na podstawie                             -> 183946.53717840125
dz u                                     -> 175771.27731131375
ust pkt                                  -> 164710.64579160677
zamówień publicznych                     -> 146950.32564421277
ustawy pzp                               -> 139073.39478404194
ust ustawy                               -> 128865.58664232567
zgodnie z                                -> 113659.92919161117
sąd najwyższy                            -> 108129.61550581329
akt kio                                  -> 107224.547713375